### Hierarchical Indexing
enable to have multiple (2 or more) index levels on an axis

In [2]:
import pandas as pd
import numpy as np

data = pd.Series(np.random.randn(9),
                index =[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                       [1, 2, 3, 1, 3, 1, 2, 2, 3]])

data

a  1    0.695853
   2    0.635431
   3    0.547703
b  1    0.588688
   3   -1.193241
c  1   -0.889712
   2   -0.858135
d  2   -0.426188
   3   -0.377636
dtype: float64

In [3]:
# this is a Series with a MultiIndex as index

data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
# partial indexing
# choose subsets of data

data['b']

1    0.588688
3   -1.193241
dtype: float64

In [5]:
data['b':'c']

b  1    0.588688
   3   -1.193241
c  1   -0.889712
   2   -0.858135
dtype: float64

In [6]:
data.loc[['b', 'd' ]]

b  1    0.588688
   3   -1.193241
d  2   -0.426188
   3   -0.377636
dtype: float64

In [7]:
data[['b', 'd' ]]

b  1    0.588688
   3   -1.193241
d  2   -0.426188
   3   -0.377636
dtype: float64

In [9]:
#selection from inner level

data.loc[:, 2]

a    0.635431
c   -0.858135
d   -0.426188
dtype: float64

In [10]:
#reshape data and group-based operations like a pivot table
data.unstack()

,1,2,3
a,0.695853,0.635431,0.547703
b,0.588688,NaN,-1.193241
c,-0.889712,-0.858135,NaN
d,NaN,-0.426188,-0.377636


In [11]:
#the inverse operation of unstack is stack

data.unstack().stack()

a  1    0.695853
   2    0.635431
   3    0.547703
b  1    0.588688
   3   -1.193241
c  1   -0.889712
   2   -0.858135
d  2   -0.426188
   3   -0.377636
dtype: float64

### DataFrame
both index and columns have hierarchical index


In [12]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index = [['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns = [['Ohio', 'Ohio', 'Colorado'],
                               ['Green', 'Red', 'Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [15]:
#assign names to the hierarchical levels to have names

frame.index.names = ['key1', 'key2']
frame

Ohio     Colorado
          Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [16]:
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [17]:
# partial column indexing
# select groups of columns

frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### Reordering and Sorting levels
- swaplevel 
- sort_index

In [19]:
frame.swaplevel('key1', 'key2')


state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [20]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [21]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by level
- aggregate by level on rows or columns

In [23]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [24]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrames columns
- convert column into index
- use set_index


In [26]:
frame = pd.DataFrame({'a':range(7), 'b':range(7, 0, -1),
                     'c': ['one', 'one', 'one', 'two', 'two',
                           'two', 'two'],
                     'd': [0, 1, 2, 0, 1, 2, 3]})

frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [30]:
#use set_index
# convert columns into index

frame2 = frame.set_index(['c', 'd'])

frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [33]:
#By default, the columns are removed from DataFrame
# leave the column in by drop=False

frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [34]:
# reset_index, doest the opposite of set_index
# convert the index back to the column

frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### 8.2 Combining and Merging Datasets

- pandas.merge - connect rows in dataframes based on one or more keys
- pandas.concat - stack objects along an axis
- combine_first, splice together overlapping data, fill in missing values

### Database-style DataFrame Joins
- merge or join combine datasets by linking rows using one or more keys

In [35]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df2 = pd.DataFrame({'key':['a', 'b', 'd'],
                    'data2': range(3)})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [36]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [37]:
# many-to-one joing
# merge to combine both dataframes

pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [38]:
# specify the overlapping column names as key
# use merge

pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [40]:
# columns names are different in each object
# specify them separately

df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [41]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [43]:
#many-to-many merges

df1 = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})

df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [44]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [45]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [46]:
# many-to-many
# how = 'inner'

pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [47]:
# merge with multiple keys
# pass a list of column names

left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})

right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

pd.merge(left, right, on=['key1', 'key2'], how='outer')



,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [49]:
#merge the dataframes
#when there are overlapping names, merge has suffixes option

pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [50]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

- merge keys can be found on index
- pass left_index = True or right_index=True (or both)
- to indicate that the index should be used as a merge key

In [51]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})

right1 = pd.DataFrame({'group_val':[3.5, 7]},
                      index =['a', 'b'])

left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [52]:
right1

,group_val
a,3.5
b,7.0


In [53]:
pd.merge(left1, right1, left_on='key', right_index=True)


,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [54]:
# how = 'outer'
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [55]:
# hierarchically indexed data
# joining on index is implicitly a multiple-key merge

lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6,2)),
                      index =[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                              [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns = ['event1', 'event2'])

lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [56]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [57]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [58]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
        right_index=True, how='outer')


,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [59]:
# use indexes of both sides of merger possible
# use left_index and right_index together

left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index = ['a', 'c', 'e'],
                    columns = ['Ohio', 'Nevada'])

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                      index=['b', 'c', 'd', 'e'],
                     columns = ['Missouri', 'Alabama'])

left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [60]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [61]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [62]:
# join instance for merging by index

left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [64]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [65]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index = ['a', 'c', 'e', 'f'],
                       columns = ['New York', 'oregon'])

another

,New York,oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [66]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [67]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### Concatenating Along an Axis
- concatenate function

In [68]:
arr = np.arange(12).reshape((3, 4))

arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [69]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [1]:
import pandas as pd
import numpy as np

s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [2]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [3]:
# By default concat works along axis = 0, produce a series
# if use axis = 1, produce a DataFrame

pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [5]:
s4 = pd.concat([s1, s3])

s4

a    0
b    1
f    5
g    6
dtype: int64

In [6]:
pd.concat([s1, s4], axis=1 )

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [7]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [14]:
# specify axes to be used with other axes using join_axes

pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

TypeError: concat() got an unexpected keyword argument 'join_axes'

In [15]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [16]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [17]:
#combine Series along axis=1, keys become the DataFrame column header

pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [18]:
# how to combine for DataFrame objects

df1 = pd.DataFrame(np.arange(6).reshape(3,2), index=['a', 'b', 'c'],
                   columns = ['one', 'two'])

df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index=['a', 'c'],
                   columns=['three', 'four'])

In [19]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [20]:
df2

,three,four
a,5,6
c,7,8


In [21]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [22]:
#pass dictionary of objects instead of list

pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [23]:
#use ignore_index=True to not overlap the index

df1 = pd.DataFrame(np.random.randn(3,4), columns=['a', 'b', 'c', 'd'])

df2 = pd.DataFrame(np.random.randn(2,3), columns=['b', 'd', 'a'])

df1

,a,b,c,d
0,0.659063,-1.078160,-0.153874,-0.275326
1,-1.272935,0.891149,-0.128726,0.688617
2,0.955463,0.410735,1.059410,-0.197397


In [24]:
df2

,b,d,a
0,0.165331,-1.872117,0.818566
1,-0.226543,1.003139,-0.837113


In [25]:
# use ignore_index= True

pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.659063,-1.078160,-0.153874,-0.275326
1,-1.272935,0.891149,-0.128726,0.688617
2,0.955463,0.410735,1.059410,-0.197397
3,0.818566,0.165331,NaN,-1.872117
4,-0.837113,-0.226543,NaN,1.003139


In [26]:
# without ignore_index =True
pd.concat([df1, df2])

,a,b,c,d
0,0.659063,-1.078160,-0.153874,-0.275326
1,-1.272935,0.891149,-0.128726,0.688617
2,0.955463,0.410735,1.059410,-0.197397
0,0.818566,0.165331,NaN,-1.872117
1,-0.837113,-0.226543,NaN,1.003139


### Combining Data with Overlap

use Numpy's where function

In [27]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index = ['f', 'e', 'd', 'c', 'b', 'a'])

b = pd.Series(np.arange(len(a), dtype=np.float64),
              index = ['f', 'e', 'd', 'c', 'b', 'a'])


In [28]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [30]:
b[-1] = np.nan
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [33]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [34]:
pd.isnull(a)

f     True
e    False
d     True
c    False
b    False
a     True
dtype: bool

In [32]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [35]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [36]:
# For DataFrames
# combine_first does the same but combine but by column

df1 = pd.DataFrame({'a': [1, np.nan, 5. , np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})

df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [37]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [38]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


### 8.3 Reshaping and Pivoting
- Re-arranging tabular data
- Reshape or pivot operations
- use stack , convert columns to row
- use unstack, convert row to column

In [39]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                    index = pd.Index(['Ohio', 'Colorado'], name = 'state'),
                    columns = pd.Index(['one', 'two', 'three'],
                    name = 'number'))

data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [40]:
# use stack to convert columns into rows

result = data.stack()

result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [41]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [42]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [43]:
# same effect

result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [44]:
# unstack can show all the missing data
# stack dont show the missing data

s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

data2 = pd.concat([s1, s2], keys=['one', 'teo'])

data2

one  a    0
     b    1
     c    2
     d    3
teo  c    4
     d    5
     e    6
dtype: int64

In [45]:
#unstack shows the NaN

data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
teo,NaN,NaN,4.0,5.0,6.0


In [46]:
# go back to convert to row

data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
teo  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [47]:
# show NaN, by showing dropna = False

data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
teo  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [49]:
# unstack in a DataFrame
# level unstacked becomes the lowest level in the result

df = pd.DataFrame({'left':result, 'right':result+5},
                   columns=pd.Index(['left', 'right'], 
                   name = 'side'))

df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [50]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [51]:
# Indicate the names of axis to unstack or stack

df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting 'Long' to 'wide' Format

- store multiple time series in databases & CSV
- do some wrangling and other data cleaning

In [52]:
data = pd.read_csv('macrodata.csv')

In [53]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [54]:
data.describe()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
count,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000
mean,1983.876847,2.492611,7221.171901,4825.293103,1012.863862,663.328640,5310.540887,105.075788,667.927586,5.311773,5.884729,239.724153,3.961330,1.336502
std,14.686817,1.118563,3214.956044,2313.346192,585.102267,140.863655,2423.515977,61.278878,455.346381,2.803071,1.458574,37.390450,3.253216,2.668799
min,1959.000000,1.000000,2710.349000,1707.400000,259.764000,460.400000,1886.900000,28.980000,139.600000,0.120000,3.400000,177.146000,-8.790000,-6.790000
25%,1971.000000,1.500000,4440.103500,2874.100000,519.147500,527.959500,3276.950000,41.050000,228.650000,3.515000,4.900000,208.631000,2.270000,-0.085000
50%,1984.000000,2.000000,6559.594000,4299.900000,896.210000,662.412000,4959.400000,104.100000,540.900000,5.010000,5.700000,236.348000,3.240000,1.340000
75%,1996.500000,3.000000,9629.346500,6398.150000,1436.681500,773.049000,6977.850000,159.650000,1102.100000,6.665000,6.800000,271.721500,4.975000,2.630000
max,2009.000000,4.000000,13415.266000,9363.600000,2264.721000,1044.088000,10077.500000,218.610000,1673.900000,15.330000,10.700000,308.013000,14.620000,10.950000


In [55]:
periods = pd.PeriodIndex(year =data.year, quarter=data.quarter, name='date')

In [56]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name = 'item')

In [57]:
data = data.reindex(columns = columns)

In [59]:
data.index = periods.to_timestamp('D', 'end')

In [61]:
ldata = data.stack().reset_index().rename(columns ={0: 'value'})

In [62]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [63]:
pivoted = ldata.pivot('date', 'item', 'value')

pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [64]:
ldata['value2'] = np.random.randn(len(ldata))

In [65]:
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.707613
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.070022
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.777333
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.101146
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.550129
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.684528
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.188229
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.236942
8,1959-09-30 23:59:59.999999999,unemp,5.300,-3.157262
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.231039


In [67]:
# Obtain DataFrame with hierarchical columns

pivoted = ldata.pivot('date', 'item')

pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.070022 -1.707613   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.550129 -0.101146   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.236942  0.188229   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.200854  0.231039   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  2.172164 -1.765257   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.777333  
1959-06-30 23:59:59.999999999  0.684528  
1959-09-30 23:59:59.999999999 -3.157262  
1959-12-31 23:59:59.999999999 -0.349735  
1960-03-31 23:59:59.999999999  0.117509

In [68]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


### Pivoting 'Wide' to 'Long' Format
invest of pivot is pd.melt

In [70]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                  'A': [1, 2, 3],
                  'B': [4, 5, 6],
                  'C': [7, 8, 9]})

df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [72]:
melted = pd.melt(df, ['key'])

melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [74]:
reshaped = melted.pivot('key', 'variable', 'value')

reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [75]:
# move data back into a column

reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [76]:
#specify subset of columns to use as value columns
# in this case use A and B

pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [77]:
# can be used without any group identifiers

pd.melt(df, value_vars=['A', 'B', 'C'])


,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [78]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
